# VIETNAMESE HATE AND OFFENSIVE SPEECH DETECTOR
## (Data Preparation)

### Download VnCoreNLP

In [ ]:
!mkdir -p models/vncorenlp/models/wordsegmenter
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
!mv VnCoreNLP-1.1.1.jar models/vncorenlp/
!mv vi-vocab models/vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr models/vncorenlp/models/wordsegmenter/

#### Check JAVA runtime
Since **VnCoreNLP requires JAVA** to run => make sure Java Runtime is already install under the working environment.

In [1]:
import os
print("JAVA_HOME:", os.getenv("JAVA_HOME"))
print("PATH:", os.getenv("PATH"))

JAVA_HOME: /usr/local/opt/openjdk
PATH: /usr/local/opt/openjdk/bin:/Users/trinb/opt/miniconda3/envs/base/bin:/usr/local/opt/openjdk/bin:/usr/local/opt/openjdk/bin:/Users/trinb/opt/miniconda3/bin:/Users/trinb/opt/miniconda3/condabin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/local/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/appleinternal/bin


### Import Libraries

In [5]:
# data manipulation, visualization
import pandas as pd
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import importlib

# modelling
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score, recall_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, BertTokenizer, BertForSequenceClassification

# custom modules
import utils.preprocess as preprocessor

In [75]:
importlib.reload(preprocessor)

<module 'utils.preprocess' from '/Users/trinb/works/vietnamese-hate-speech-detection/utils/preprocess.py'>

## I. Data Overview

In [3]:
# load data
train = pd.read_csv("data/vihsd/train.csv", index_col=False)
dev = pd.read_csv("data/vihsd/dev.csv", index_col=False)
test = pd.read_csv("data/vihsd/test.csv", index_col=False)

train.reset_index(drop=True)

print(len(train))
print(len(dev))
print(len(test))
     

24048
2672
6680


In [30]:
train.head(15)

,free_text,label_id
0,Em được làm fan cứng luôn rồi nè ❤️ reaction q...,0
1,Đúng là bọn mắt híp lò xo thụt :))) bên việt n...,2
2,Đậu Văn Cường giờ giống thằng sida hơn à,0
3,CÔN ĐỒ CỤC SÚC VÔ NHÂN TÍNH ĐỀ NGHI VN. NHÀ NƯ...,2
4,Từ lý thuyết đến thực hành là cả 1 câu chuyện ...,0
5,Coronavirus is manmade,0
6,Đố chúng m nhận ra ai,0
7,Lúp lúp như chó .,1
8,🥰🥰🥰,0
9,"Thế mà mình nói mấy thằng bắc kì, bọn đó lại b...",2


# II. Data Preprocess

### Examine tokenizer

#### vncorenlp

In [12]:
preprocessor.tokenizer(train['free_text'][8])

[['🥰🥰🥰']]

#### ViTokenizer

In [11]:
preprocessor.tokenizer(train['free_text'][8], option=2)

'🥰 🥰 🥰'

### Examine filter stopwords

In [15]:
preprocessor.filter_stop_words(train['free_text'][4])

'Từ lý thuyết thực hành 1 câu =))'

### Examine deEmojify

In [60]:
preprocessor.deEmojify(train['free_text'][1])

'Đúng là bọn mắt híp lò xo thụt  bên việt nam t cái này ra cách đây 10 năm r và bọn t gọi là cái L '

In [68]:
preprocessor.deEmojify('aaa =]]]] aa')

'aaa  aa'

### Prepare data

In [46]:
X_train = train['free_text']
y_train = train['label_id'].values

X_dev = dev['free_text']
y_dev = dev['label_id'].values

X_test = test['free_text']
y_test = test['label_id'].values

In [76]:
train_X, train_y = preprocessor.pre_process_features(X_train, y_train, tokenized=True, lowercased = False)

0        Em được làm fan cứng luôn rồi nè ❤️ reaction q...
1        Đúng là bọn mắt híp lò xo thụt :))) bên việt n...
2                 Đậu Văn Cường giờ giống thằng sida hơn à
3        CÔN ĐỒ CỤC SÚC VÔ NHÂN TÍNH ĐỀ NGHI VN. NHÀ NƯ...
4        Từ lý thuyết đến thực hành là cả 1 câu chuyện ...
                               ...                        
24043                    Thả tim cho sự cống hiến của ngài
24044                                               Thua !
24045    Ở Việt Nam thì có khánh sky..hải ngoại có trần...
24046      1:22 ai lớp du chu cờ moo, Duyên dáng quá cô ơi
24047    Cụ này mắt mới cắt mý sao ta kkk đấy vnch đấy ...
Name: free_text, Length: 24048, dtype: object
Em fan cứng nè reaction cute coi mấy hợp_lí
Đúng bọn mắt híp lò_xo thụt việt_nam t 10 r bọn t gọi L
Đậu_Văn_Cường thằng sida
CÔN_ĐỒ CỤC_SÚC VÔ_NHÂN TÍNH ĐỀ NGHI VN .NHÀ_NƯỚC VN BAN THƯỞNG .
Từ lý_thuyết thực_hành 1 câu
Coronavirus is manmade
Đố m
Lúp lúp chó .

Thế mấy thằng bắc kì , bọn bâu vô phân_biệt

In [90]:
val_X, val_y = preprocessor.pre_process_features(X_dev, y_dev, tokenized=True, lowercased = False)

0                                            Coi cười xỉu
1                         Chi ba vang ngoc dep va tre mai
2       Chần vần một đống, không ai đoán trước được đừ...
3                                   Du học sinh Úc mà lại
4       Dm báo lá cải.. Nghĩ gì khi ở qn mà có hơn 30%...
                              ...                        
2667                      Thật sự là đã hi sinh quá nhiều
2668                                  chật chội vậy cơ mà
2669                            Chống dịch như chống giặc
2670    Bao nhiêu ae ra đảo giờ chưa về hôm bữa theo 1...
2671    Dcm ăn pải lốp lái thì giết con nhà ng ta ah c...
Name: free_text, Length: 2672, dtype: object
Coi cười xỉu
Chi vang ngoc dep va tre mai
Chần vần đống , đoán đừng xưng_hô tiên_tri
Du_học_sinh Úc
Dm báo lá_cải ..Nghĩ qn 30% hộ nghèo ..Lên google tỉnh kinh_tế
Mai_Đoàn nãy rep cmt rep xong tr 4G quẹt
My su
Muốn 1 tộc hợp_tác sản_phẩm cạnh_tranh Tùng
gays nn
@ Ngoc_Quynh_Nhu_Nguyen Nó góp ngon thẳng Facebook Lind

### Export csv

In [86]:
data = { 'text': train_X, 'labels': train_y }

In [87]:
train_df = pd.DataFrame(data)

In [88]:
train_df

,text,labels
0,Em fan cứng nè reaction cute coi mấy hợp_lí,0
1,Đúng bọn mắt híp lò_xo thụt việt_nam t 10 r bọ...,2
2,Đậu_Văn_Cường thằng sida,0
3,CÔN_ĐỒ CỤC_SÚC VÔ_NHÂN TÍNH ĐỀ NGHI VN .NHÀ_NƯ...,2
4,Từ lý_thuyết thực_hành 1 câu,0
...,...,...
24043,Thả tim cống_hiến ngài,0
24044,Thua !,0
24045,Ở Việt_Nam khánh sky .. hải_ngoại trần dần .. ...,0
24046,"122 lớp du chu cờ moo , Duyên_dáng",0


In [89]:
train_df.to_csv('data/processed_train.csv')

#### Val dataset

In [91]:
val_data = { 'text': val_X, 'labels': val_y }

In [92]:
val_df = pd.DataFrame(val_data)

In [93]:
val_df.to_csv('data/processed_val.csv')

In [99]:
val_df['labels'].value_counts()

labels
0    2190
2     270
1     212
Name: count, dtype: int64

In [100]:
val_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2672 entries, 0 to 2671
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    2672 non-null   object
 1   labels  2672 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 41.9+ KB
